## todo:
* * _Catagorical Dummy Encoding?__
    * Anything here? Check for catagorical using unique


# Data Cleaning Notebook


In [1]:
# Import the required libraries
import pandas as pd
import numpy as np

# set up pandas to display floats in a more human friendly way
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
king_data_raw = pd.read_csv('../data/kc_house_data.csv')
king_data_raw.head(10)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,"221,900.00",3,1.00,1180,5650,1.00,nan,0.00,...,7,1180,0.0,1955,0.00,98178,47.51,-122.26,1340,5650
1,6414100192,12/9/2014,"538,000.00",3,2.25,2570,7242,2.00,0.00,0.00,...,7,2170,400.0,1951,"1,991.00",98125,47.72,-122.32,1690,7639
2,5631500400,2/25/2015,"180,000.00",2,1.00,770,10000,1.00,0.00,0.00,...,6,770,0.0,1933,nan,98028,47.74,-122.23,2720,8062
3,2487200875,12/9/2014,"604,000.00",4,3.00,1960,5000,1.00,0.00,0.00,...,7,1050,910.0,1965,0.00,98136,47.52,-122.39,1360,5000
4,1954400510,2/18/2015,"510,000.00",3,2.00,1680,8080,1.00,0.00,0.00,...,8,1680,0.0,1987,0.00,98074,47.62,-122.05,1800,7503
5,7237550310,5/12/2014,"1,230,000.00",4,4.50,5420,101930,1.00,0.00,0.00,...,11,3890,1530.0,2001,0.00,98053,47.66,-122.00,4760,101930
6,1321400060,6/27/2014,"257,500.00",3,2.25,1715,6819,2.00,0.00,0.00,...,7,1715,?,1995,0.00,98003,47.31,-122.33,2238,6819
7,2008000270,1/15/2015,"291,850.00",3,1.50,1060,9711,1.00,0.00,nan,...,7,1060,0.0,1963,0.00,98198,47.41,-122.31,1650,9711
8,2414600126,4/15/2015,"229,500.00",3,1.00,1780,7470,1.00,0.00,0.00,...,7,1050,730.0,1960,0.00,98146,47.51,-122.34,1780,8113
9,3793500160,3/12/2015,"323,000.00",3,2.50,1890,6560,2.00,0.00,0.00,...,7,1890,0.0,2003,0.00,98038,47.37,-122.03,2390,7570


In [3]:
king_data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  float64
 9   view           21534 non-null  float64
 10  condition      21597 non-null  int64  
 11  grade          21597 non-null  int64  
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

In [4]:
# Make a deep copy and do procesing on this dataframe moving forward
# Deep copy has own copy of data and index.
king_processed = king_data_raw.copy(deep=True)

# Original Data Column Descriptions

* id            - Unique identifier for house
* date          - Date house was sold
* price         - Sales price of home - __this is the prediction target__
* bedrooms      - Number of bedrooms
* bathrooms     - Number of bathrooms
* sqft_living   - Square footage of living area
* sqft_lot      - Square footage of  lot
* floors        - # floors (levels) in house
* waterfront    - Number from 0 to 1 reflecting if house is categorized as a waterfront property - Categorical
* view          - Number from 0 to 4 reflecting quality of sceanic view - Categorical
* condition     - Number from 1 to 5 reflecting the overall condition of the property - Categorical
* grade         - Number from 1 to 13 refelecting the overall grade given to the property, based on King County grading system
* sqft_above    - Square footage of living area above ground/ (is this sqft_living - sqft_basement?) from basement
* sqft_basement - Square footage of the basement
* yr_built      - Year house was built
* yr_renovated  - Year house was renovated
* zipcode       - zip code
* lat           - Latitude coordinate
* long          - Longitude coordinate
* sqft_living15 - Average square footage of living area of the nearest 15 neighbors 
* sqft_lot15    - Average square footage of the  lots of the nearest 15 neighbors 
    * Thanks to Data Science cohort members past and present for Data Column Descriptions

__Grade ratings from [King County] (https://info.kingcounty.gov/assessor/esales/Glossary.aspx?type=r#building%20grade)__

Represents the construction quality of improvements. Grades run from grade 1 to 13. Generally defined as:

1-3 __Falls short of minimum building standards.__ Normally cabin or inferior structure.

4 Generally older, low quality construction. __Does not meet code.__

5 Low construction costs and workmanship. Small, simple design.

6 __Lowest grade currently meeting building code.__ Low quality materials and simple designs.

7 __Average grade of construction and design.__ Commonly seen in plats and older sub-divisions.

8 Just above average in construction and design. Usually better materials in both the exterior and interior finish work.

9 Better architectural design with extra interior and exterior design and quality.

10 Homes of this quality generally have __high quality features__. Finish work is better and more design quality is seen in the floor plans. Generally have a larger square footage.

11 Custom design and higher quality finish work with added amenities of solid woods, bathroom fixtures and more luxurious options.

12 Custom design and excellent builders. All materials are of the highest quality and all conveniences are present.

13 Generally custom designed and built. __Mansion level.__ Large amount of highest quality cabinet work, wood trim, marble, entry ways etc.

# Data Preparation

## Data cleaning steps:
* __Duplicate check__ 
    * Absolute duplicates (checking all columns) - no duplicates found
    * Check using id, the house unique identifier -  duplicate ids _were_ found. 
        * Decided to keep the last duplicate row found (making assumption that records are most likely in order by sale date).
    * waterfront 
    * view
    * yr_renovated 
* __Address Null Values (nans)__
    * waterfront
    * view
* __Data type conversions__
    * sqft_basement - convert from string to int
    * waterfront - convert from float to int
    * view - convert from float to int
    * yr_renovated - convert from float to int
* __New column creation__
    * date - split out the Month, Day, Year for potential feature use
    * yr_renovated - create a new catagorical variable,renovated_status, based off logic detailed in section below.
    * sqft_living, sqft_living15, sqft_lot and sqft_lot15 - create two new catagorical variables, sqft_living15_different and sqft_lot15_different. More details below on reason.
    * scenic_status - create a new catagorical variable, scenic_status, based off logic detailed in section below.
* __Column rename__
    * view - renamed to view_rank as the original col name was 1) confusing and 2) caused a plotting problem later on. :/
* _Catagorical Dummy Encoding?__
    * Anything here? Check for catagorical using unique


## Duplicate check - Absolute duplicates

In [5]:
# Duplicate check
king_dups = king_processed[king_processed.duplicated()]
print(len(king_dups))

0


## Duplicate check - House ID duplicates

In [6]:
# We suspect there are duplicate house ids...check it out
print(king_processed.shape)
print(len(king_processed['id'].unique()))

(21597, 21)
21420


In [7]:
# Save all the duplicated house id records to a file for inspection if needed.
dup_id_series = king_processed.duplicated(subset=['id'], keep=False)
dup_house_ids = king_processed[dup_id_series]
dup_house_ids.to_csv('../data/dup_house_ids.csv', index=True)

# There are 353 total duplicates based on house id
print(dup_house_ids.shape)
dup_house_ids.head(6)

(353, 21)


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
93,6021501535,7/25/2014,"430,000.00",3,1.50,1580,5000,1.00,0.00,0.00,...,8,1290,290.0,1939,0.00,98117,47.69,-122.39,1570,4500
94,6021501535,12/23/2014,"700,000.00",3,1.50,1580,5000,1.00,0.00,0.00,...,8,1290,290.0,1939,0.00,98117,47.69,-122.39,1570,4500
313,4139480200,6/18/2014,"1,380,000.00",4,3.25,4290,12103,1.00,0.00,3.00,...,11,2690,1600.0,1997,0.00,98006,47.55,-122.10,3860,11244
314,4139480200,12/9/2014,"1,400,000.00",4,3.25,4290,12103,1.00,0.00,3.00,...,11,2690,1600.0,1997,0.00,98006,47.55,-122.10,3860,11244
324,7520000520,9/5/2014,"232,000.00",2,1.00,1240,12092,1.00,nan,0.00,...,6,960,280.0,1922,"1,984.00",98146,47.50,-122.35,1820,7460
325,7520000520,3/11/2015,"240,500.00",2,1.00,1240,12092,1.00,0.00,0.00,...,6,960,280.0,1922,"1,984.00",98146,47.50,-122.35,1820,7460


In [8]:
# Remove duplicates and keep last occurences
king_processed.drop_duplicates(subset=['id'], keep='last', inplace=True)
print(king_processed.shape)
print(len(king_processed['id'].unique()))

(21420, 21)
21420


## Address Null Values (nans) in our data

In [9]:
# Address nans for waterfront column
print(king_processed.waterfront.unique())
print(king_processed['waterfront'].isna().sum())

[nan  0.  1.]
2353


In [10]:
# Assuming that, since waterfront property is desirable real estate, it would be marked as such. 
# Setting waterfront nan values to 0 
king_processed['waterfront'].fillna(0.0, inplace=True)
print(king_processed['waterfront'].isna().sum())

0


In [11]:
# Address nans for view column 
print(king_processed.view.unique())
print(king_processed['view'].isna().sum())

[ 0. nan  3.  4.  2.  1.]
63


In [12]:
# Assuming that, since having a high rated view attribute would indicate desirable real estate, a good view would be marked as such. 
# Setting view nan values to 0 
king_processed['view'].fillna(0.0, inplace=True)
print(king_processed['view'].isna().sum())

0


In [13]:
# Address nans for yr_renovated column
print(king_processed.yr_renovated.unique())
print(king_processed['yr_renovated'].isna().sum())

[   0. 1991.   nan 2002. 2010. 1992. 2013. 1994. 1978. 2005. 2003. 1984.
 1954. 2014. 2011. 1983. 1945. 1990. 1988. 1977. 1981. 1995. 2000. 1999.
 1998. 1970. 1989. 2004. 1986. 2007. 1987. 2006. 1985. 2001. 1980. 1971.
 1979. 1997. 1950. 1969. 1948. 2009. 2015. 1974. 2008. 1968. 2012. 1963.
 1951. 1962. 1953. 1993. 1996. 1955. 1982. 1956. 1940. 1976. 1946. 1975.
 1964. 1973. 1957. 1959. 1960. 1967. 1965. 1934. 1972. 1944. 1958.]
3813


__yr_renovated__ column/feature feature needs some more thought applied to how to handle it. 
I'll start by setting the nans to 0.  
I might need to add a renovated_status categorical variable.
Here's my thoughts on some valid assumptions around renovated_status.
* Recently constructed homes have probably not been renovated, so even if the __yr_renovated__ was nan/0, I will assume home wasn't renovated.
* __Also worth noting__ that if there is a difference, I might need to use the "2015" columns/features instead of the "original" features.  Could do two different models/split out the data.




In [14]:
# Setting the nans to 0. Will perform other checks to set a new categorical variable, renovated_status, later on.
king_processed['yr_renovated'].fillna(0, inplace=True)
print(king_processed['yr_renovated'].isna().sum())

0


## Data type conversions

In [15]:
#no issues with simply changing waterfront to int64
king_processed['waterfront'] = np.int64(king_processed['waterfront'])

In [16]:
#no issues with simply changing view to int64
king_processed['view'] = np.int64(king_processed['view'])

In [17]:
#no issues with simply changing yr_renovated to int64
king_processed['yr_renovated'] = np.int64(king_processed['yr_renovated'])

In [18]:
# sqft_basement needed more wrangling 
# Was a String with either a non-numeric value (like ?) or a String representation of a float 400.00 with unneeded decimal places :/
def convert_sqft_basement(some_string, default):
    result = np.int64(default)
    try:
        result = np.int64(float(some_string))
    except (ValueError, TypeError):
        print(f'could not convert {some_string}, using {result}')
    finally:
        return result

king_processed['sqft_basement'] = king_processed.apply(lambda row: convert_sqft_basement(row.sqft_basement, 0), axis=1)

could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not conv

could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not convert ?, using 0
could not conv

In [19]:
print(king_processed.dtypes)
king_processed.head(10)

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,"221,900.00",3,1.00,1180,5650,1.00,0,0,...,7,1180,0,1955,0,98178,47.51,-122.26,1340,5650
1,6414100192,12/9/2014,"538,000.00",3,2.25,2570,7242,2.00,0,0,...,7,2170,400,1951,1991,98125,47.72,-122.32,1690,7639
2,5631500400,2/25/2015,"180,000.00",2,1.00,770,10000,1.00,0,0,...,6,770,0,1933,0,98028,47.74,-122.23,2720,8062
3,2487200875,12/9/2014,"604,000.00",4,3.00,1960,5000,1.00,0,0,...,7,1050,910,1965,0,98136,47.52,-122.39,1360,5000
4,1954400510,2/18/2015,"510,000.00",3,2.00,1680,8080,1.00,0,0,...,8,1680,0,1987,0,98074,47.62,-122.05,1800,7503
5,7237550310,5/12/2014,"1,230,000.00",4,4.50,5420,101930,1.00,0,0,...,11,3890,1530,2001,0,98053,47.66,-122.00,4760,101930
6,1321400060,6/27/2014,"257,500.00",3,2.25,1715,6819,2.00,0,0,...,7,1715,0,1995,0,98003,47.31,-122.33,2238,6819
7,2008000270,1/15/2015,"291,850.00",3,1.50,1060,9711,1.00,0,0,...,7,1060,0,1963,0,98198,47.41,-122.31,1650,9711
8,2414600126,4/15/2015,"229,500.00",3,1.00,1780,7470,1.00,0,0,...,7,1050,730,1960,0,98146,47.51,-122.34,1780,8113
9,3793500160,3/12/2015,"323,000.00",3,2.50,1890,6560,2.00,0,0,...,7,1890,0,2003,0,98038,47.37,-122.03,2390,7570


## Column renaming

In [20]:
# the column name view is confusing and caused a problem with plotting. Rename it.     
king_processed.rename(columns={'view':'view_rank'}, inplace=True)

## New columns creation

In [21]:
# The date column is not really useful. Pull out the month, day, year values from this date String column and save a new cols.
def get_date_element(date_as_string, desired_index):
    split_vals = date_as_string.split('/')
    if len(split_vals) != 3:
        split_vals = [0, 0, 0]
    return np.int64(split_vals[desired_index])


In [22]:
king_processed['date_month'] = king_processed.apply(lambda row: get_date_element(row.date, 0), axis=1)
king_processed['date_day'] = king_processed.apply(lambda row: get_date_element(row.date, 1), axis=1)
king_processed['date_year'] = king_processed.apply(lambda row: get_date_element(row.date, 2), axis=1)

In [23]:
# Make a categorical variable to reflect the renovated status. 
# Start by setting value Y if there is a valid yr_renovated year (>0) and UNK if it is 0
king_processed['renovated_status'] = king_processed.apply(lambda row: 'Y' if row.yr_renovated > 0 else 'UNK', axis=1)


In [24]:
# Where the yr_renovated is provided, what is the most recent yr_built?
print(king_processed[(king_processed['yr_renovated']>0)].yr_built.max())

2003


For the set of homes with a valid __yr_renovated__, the most recent yr_built was 2003.  I will assume that any home build after 2003 with __yr_renovated__ of 0 was __not renovated__ and update the __renovated_status__ from 'UNK' to 'N'. 

In [25]:
print(king_processed[king_processed['renovated_status'] == 'UNK'].shape)
print(king_processed[king_processed['renovated_status'] == 'N'].shape)

(20680, 25)
(0, 25)


In [26]:
def check_for_recent_homes(yr_renovated, yr_built):
    return (yr_renovated == 0 and yr_built > 2003)

king_processed['renovated_status'] = king_processed.apply(lambda row: 'N' if check_for_recent_homes(row.yr_renovated, row.yr_built) else row.renovated_status, axis=1)
king_processed.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view_rank,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,date_month,date_day,date_year,renovated_status
0,7129300520,10/13/2014,"221,900.00",3,1.00,1180,5650,1.00,0,0,...,0,98178,47.51,-122.26,1340,5650,10,13,2014,UNK
1,6414100192,12/9/2014,"538,000.00",3,2.25,2570,7242,2.00,0,0,...,1991,98125,47.72,-122.32,1690,7639,12,9,2014,Y
2,5631500400,2/25/2015,"180,000.00",2,1.00,770,10000,1.00,0,0,...,0,98028,47.74,-122.23,2720,8062,2,25,2015,UNK
3,2487200875,12/9/2014,"604,000.00",4,3.00,1960,5000,1.00,0,0,...,0,98136,47.52,-122.39,1360,5000,12,9,2014,UNK
4,1954400510,2/18/2015,"510,000.00",3,2.00,1680,8080,1.00,0,0,...,0,98074,47.62,-122.05,1800,7503,2,18,2015,UNK


In [27]:
print(king_processed[king_processed['renovated_status'] == 'UNK'].shape)
print(king_processed[king_processed['renovated_status'] == 'N'].shape)

(17099, 25)
(3581, 25)


From the data description, the _15 columns contain avg sqft from 15 nearest neighbors (living area and lot size). This would indicate if a house was smaller/similar/larger than its neighbors. 
* sqft_living15 - the average house square footage of the 15 closest houses
* sqft_lot15 - the average lot square footage of the 15 closest houses

Creating create new columns to capture differences between the sqft_living and sqft_lot and their corresponding neighbors values


In [28]:
king_processed['sqft_living15_diff'] = king_processed.apply(lambda row: row.sqft_living - row.sqft_living15, axis=1)
king_processed['sqft_lot15_diff'] = king_processed.apply(lambda row: row.sqft_lot - row.sqft_lot15, axis=1)

In [29]:
king_processed.head(10)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view_rank,...,lat,long,sqft_living15,sqft_lot15,date_month,date_day,date_year,renovated_status,sqft_living15_diff,sqft_lot15_diff
0,7129300520,10/13/2014,"221,900.00",3,1.00,1180,5650,1.00,0,0,...,47.51,-122.26,1340,5650,10,13,2014,UNK,-160,0
1,6414100192,12/9/2014,"538,000.00",3,2.25,2570,7242,2.00,0,0,...,47.72,-122.32,1690,7639,12,9,2014,Y,880,-397
2,5631500400,2/25/2015,"180,000.00",2,1.00,770,10000,1.00,0,0,...,47.74,-122.23,2720,8062,2,25,2015,UNK,-1950,1938
3,2487200875,12/9/2014,"604,000.00",4,3.00,1960,5000,1.00,0,0,...,47.52,-122.39,1360,5000,12,9,2014,UNK,600,0
4,1954400510,2/18/2015,"510,000.00",3,2.00,1680,8080,1.00,0,0,...,47.62,-122.05,1800,7503,2,18,2015,UNK,-120,577
5,7237550310,5/12/2014,"1,230,000.00",4,4.50,5420,101930,1.00,0,0,...,47.66,-122.00,4760,101930,5,12,2014,UNK,660,0
6,1321400060,6/27/2014,"257,500.00",3,2.25,1715,6819,2.00,0,0,...,47.31,-122.33,2238,6819,6,27,2014,UNK,-523,0
7,2008000270,1/15/2015,"291,850.00",3,1.50,1060,9711,1.00,0,0,...,47.41,-122.31,1650,9711,1,15,2015,UNK,-590,0
8,2414600126,4/15/2015,"229,500.00",3,1.00,1780,7470,1.00,0,0,...,47.51,-122.34,1780,8113,4,15,2015,UNK,0,-643
9,3793500160,3/12/2015,"323,000.00",3,2.50,1890,6560,2.00,0,0,...,47.37,-122.03,2390,7570,3,12,2015,UNK,-500,-1010


In [30]:
# Make a categorical variable to reflect if home is considered, scenic_status, which is 'Y' if view > 1  OR waterfront == 1       

def check_for_scenic_homes(view_rank, waterfront):
    val = 'N'
    if (view_rank > 1 or waterfront == 1):
        val = 'Y'
    return val

king_processed['scenic_status'] = king_processed.apply(lambda row: check_for_scenic_homes(row.view_rank, row.waterfront), axis=1)
king_processed.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view_rank,...,long,sqft_living15,sqft_lot15,date_month,date_day,date_year,renovated_status,sqft_living15_diff,sqft_lot15_diff,scenic_status
0,7129300520,10/13/2014,"221,900.00",3,1.00,1180,5650,1.00,0,0,...,-122.26,1340,5650,10,13,2014,UNK,-160,0,N
1,6414100192,12/9/2014,"538,000.00",3,2.25,2570,7242,2.00,0,0,...,-122.32,1690,7639,12,9,2014,Y,880,-397,N
2,5631500400,2/25/2015,"180,000.00",2,1.00,770,10000,1.00,0,0,...,-122.23,2720,8062,2,25,2015,UNK,-1950,1938,N
3,2487200875,12/9/2014,"604,000.00",4,3.00,1960,5000,1.00,0,0,...,-122.39,1360,5000,12,9,2014,UNK,600,0,N
4,1954400510,2/18/2015,"510,000.00",3,2.00,1680,8080,1.00,0,0,...,-122.05,1800,7503,2,18,2015,UNK,-120,577,N


### One last look at data after processing

In [32]:
king_processed.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view_rank,condition,...,zipcode,lat,long,sqft_living15,sqft_lot15,date_month,date_day,date_year,sqft_living15_diff,sqft_lot15_diff
count,"21,420.00","21,420.00","21,420.00","21,420.00","21,420.00","21,420.00","21,420.00","21,420.00","21,420.00","21,420.00",...,"21,420.00","21,420.00","21,420.00","21,420.00","21,420.00","21,420.00","21,420.00","21,420.00","21,420.00","21,420.00"
mean,"4,580,939,869.16","541,861.43",3.37,2.12,"2,083.13","15,128.04",1.50,0.01,0.23,3.41,...,"98,077.87",47.56,-122.21,"1,988.38","12,775.72",6.56,15.69,"2,014.33",94.75,"2,352.32"
std,"2,876,761,172.45","367,556.94",0.93,0.77,918.81,"41,530.80",0.54,0.08,0.77,0.65,...,53.48,0.14,0.14,685.54,"27,345.62",3.12,8.64,0.47,601.30,"29,022.93"
min,"1,000,102.00","78,000.00",1.00,0.50,370.00,520.00,1.00,0.00,0.00,1.00,...,"98,001.00",47.16,-122.52,399.00,651.00,1.00,1.00,"2,014.00","-2,310.00","-326,879.00"
25%,"2,123,537,386.75","324,950.00",3.00,1.75,"1,430.00","5,040.00",1.00,0.00,0.00,3.00,...,"98,033.00",47.47,-122.33,"1,490.00","5,100.00",4.00,8.00,"2,014.00",-220.00,-420.25
50%,"3,904,921,185.00","450,550.00",3.00,2.25,"1,920.00","7,614.00",1.50,0.00,0.00,3.00,...,"98,065.00",47.57,-122.23,"1,840.00","7,620.00",6.00,16.00,"2,014.00",0.00,0.00
75%,"7,308,900,186.25","645,000.00",4.00,2.50,"2,550.00","10,690.50",2.00,0.00,0.00,4.00,...,"98,117.00",47.68,-122.12,"2,370.00","10,086.25",9.00,23.00,"2,015.00",310.00,680.00
max,"9,900,000,190.00","7,700,000.00",33.00,8.00,"13,540.00","1,651,359.00",3.50,1.00,4.00,5.00,...,"98,199.00",47.78,-121.31,"6,210.00","871,200.00",12.00,31.00,"2,015.00","8,690.00","1,225,778.00"


In [33]:
king_processed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21420 entries, 0 to 21596
Data columns (total 28 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  21420 non-null  int64  
 1   date                21420 non-null  object 
 2   price               21420 non-null  float64
 3   bedrooms            21420 non-null  int64  
 4   bathrooms           21420 non-null  float64
 5   sqft_living         21420 non-null  int64  
 6   sqft_lot            21420 non-null  int64  
 7   floors              21420 non-null  float64
 8   waterfront          21420 non-null  int64  
 9   view_rank           21420 non-null  int64  
 10  condition           21420 non-null  int64  
 11  grade               21420 non-null  int64  
 12  sqft_above          21420 non-null  int64  
 13  sqft_basement       21420 non-null  int64  
 14  yr_built            21420 non-null  int64  
 15  yr_renovated        21420 non-null  int64  
 16  zipc

## Save processed data to file


In [34]:
# save the processed data to file
king_processed.to_csv('../data/proc_kc_house_data.csv', index=False)

In [35]:
king_processed.shape

(21420, 28)